In [1]:
import pandas as pd
import numpy as np
import LinearModels as LM
import matplotlib.pyplot as plt
import NonLinearModels_post as nlm

from scipy.stats import norm
from scipy.stats import genextreme

%load_ext autoreload
%autoreload 2

import clogit 
import estimation as est

import logit
import estimation2 as est2

In [2]:
pd.options.display.float_format = '{:.12f}'.format
path = "/Users/Lasse/Documents/Bachelor/Seb/MotorRegisterData-main/"

In [5]:
choice_data = pd.read_csv(path + 'clogit__data_subset.csv', delimiter=',', encoding = 'unicode_escape')

In [6]:
choice_data

Unnamed: 0    ID    Fuel  Benzin  Diesel  El  \
0              0     0  Benzin       1       0   0   
1              1     1  Diesel       0       1   0   
2              2     2  Benzin       1       0   0   
3              3     3  Benzin       1       0   0   
4              4     4  Benzin       1       0   0   
...          ...   ...     ...     ...     ...  ..   
2785        3353  2785  Diesel       0       1   0   
2786        3378  2786  Benzin       1       0   0   
2787        3379  2787      El       0       0   1   
2788        3380  2788  Benzin       1       0   0   
2789        3381  2789  Benzin       1       0   0   

                                        Key  Year        Make    market_year  \
0                toyota-avensis-2006-Benzin  2006      toyota 0.033475590202   
1             volkswagen-passat-2006-Diesel  2006  volkswagen 0.033167229938   
2                  renault-clio-2006-Benzin  2006     renault 0.027863433407   
3                  suzuki-swift-2006-Benzin  2006      suzuki 0.027172706416   
4                    fiat-punto-2006-Benzin  2006        fiat 0.023083849323   
...                                     ...   ...         ...            ...   
2785               dacia-duster-2020-Diesel  2020       dacia 0.000386870922   
2786  volkswagen-golf-sportsvan-2020-Benzin  2020  volkswagen 0.003859273343   
2787                  volkswagen-up-2020-El  2020  volkswagen 0.001745637087   
2788         volkswagen-t-cross-2020-Benzin  2020  volkswagen 0.014007558137   
2789           volkswagen-t-roc-2020-Benzin  2020  volkswagen 0.015564477701   

      Weight (kg)  Engine effect (kW)   Prices (2015-DKK)  Cost/km (DKK)  \
0            1820    108.000000000000 403349.299100000004 0.813669065000   
1            2170    103.000000000000 378504.672900000005 0.681879195000   
2            1425     43.000000000000 134344.626199999999 0.677245509000   
3            1475     68.000000000000 163536.214999999997 0.734415584000   
4            1590     48.000000000000 150287.383200000011 0.689634146000   
...           ...                 ...                 ...            ...   
2785         1848     85.000000000000 170224.371400000004 0.401119403000   
2786         1930    110.000000000000 303593.810400000017 0.583653846000   
2787         1530     61.000000000000 153689.555099999998 0.306765224000   
2788         1760     85.000000000000 263941.972899999993 0.541964286000   
2789         1860    110.000000000000 254948.742700000003 0.583653846000   

           Size (m3)       Horsepower  Fuelchoice  
0    12.573440000000 139.307692300000           2  
1    12.734904000000 128.333333300000           1  
2     8.872728000000  60.000000000000           2  
3     9.379500000000  94.000000000000           2  
4     9.483248000000  63.888888890000           2  
...              ...              ...         ...  
2785 13.202280000000 103.750000000000           1  
2786 12.411532000000 150.000000000000           2  
2787  8.855775289000  61.222222220000           0  
2788 11.429088000000 115.000000000000           2  
2789 12.086802000000 150.000000000000           2  

[2790 rows x 17 columns]

In [8]:
N = choice_data.Year.nunique()
N

15

In [10]:
J = choice_data.Key.nunique()/N
J = int(J)
J

186

In [12]:
x_vars = list(choice_data.iloc[:,[10,11,12,13,14]])
# Flere forklarende variabler: Stor bil / lille bil, El dummy, 
x_vars

['Weight (kg)',
 'Engine effect (kW)',
 'Prices (2015-DKK)',
 'Cost/km (DKK)',
 'Size (m3)']

In [13]:
x_lab = ['Weight (kg)', 'Engine effect (kW)', 'Prices (2015-DKK)', 'Cost/km (DKK)', 'Horsepower']

In [14]:
K = len(x_vars)
K

5

In [30]:
y = choice_data['market_year'].values.reshape(N,J)
y

array([[0.03347559, 0.03316723, 0.02786343, ..., 0.00040704, 0.00061055,
        0.00031453],
       [0.00441973, 0.03407057, 0.02697202, ..., 0.00090724, 0.00394772,
        0.00057009],
       [0.00091579, 0.00065413, 0.00480134, ..., 0.00055601, 0.00149797,
        0.00043173],
       ...,
       [0.00388433, 0.00352854, 0.00829317, ..., 0.00783468, 0.00021274,
        0.00023841],
       [0.00249948, 0.00024223, 0.0097873 , ..., 0.0001685 , 0.00042126,
        0.0018711 ],
       [0.00538317, 0.00025005, 0.00376963, ..., 0.00174564, 0.01400756,
        0.01556448]])

In [51]:
x = choice_data[x_vars].values.reshape((N,J,K))
x[:,0,2]

array([ 403349.2991,  499915.0402, 1569643.729 ,  620178.7281,
        749195.0697,  984427.5287,  532583.5031,  322779.798 ,
        357132.5301,  526845.    ,  574285.1446,  563715.9763,
        310101.7613,  284461.165 ,  189473.8878])

In [45]:
theta0 = np.zeros((K,)) 
theta0.shape

(5,)

In [33]:
result = est2.estimate(logit.q, theta0, y, x)
result

         Current function value: 5.145144
         Iterations: 16
         Function evaluations: 426
         Gradient evaluations: 69


{'theta_hat': array([-3.74093093e-04, -3.57586490e-04, -1.63478706e-07, -3.05495733e-01,
        -1.09118506e-01]),
 'success': False,
 'nit': 16,
 'nfev': 426,
 'fun': 5.145143959173024,
 'se': array([3.28989568e-02, 2.44934789e-01, 3.45399060e-05, 1.83424086e+01,
        5.85417138e+00]),
 't': array([-0.01137097, -0.00145993, -0.00473304, -0.01665516, -0.01863945]),
 'cov': array([[ 1.08234136e-03,  1.68633810e-03, -3.91891118e-07,
          6.73378819e-02, -1.84454114e-01],
        [ 1.68633810e-03,  5.99930506e-02, -4.18429733e-06,
         -1.58973009e+00, -5.19927895e-01],
        [-3.91891118e-07, -4.18429733e-06,  1.19300510e-09,
          1.23551288e-05,  7.90533090e-05],
        [ 6.73378819e-02, -1.58973009e+00,  1.23551288e-05,
          3.36443954e+02, -1.10971220e+01],
        [-1.84454114e-01, -5.19927895e-01,  7.90533090e-05,
         -1.10971220e+01,  3.42713226e+01]])}

In [21]:
est2.print_table(x_lab, result)

Optimizer failed after 16 iter. (426 func. evals.). Final criterion:    5.145.
Results


theta              se               t
Weight (kg)        -0.000400000000  0.032900000000 -0.011400000000
Engine effect (kW) -0.000400000000  0.244900000000 -0.001500000000
Prices (2015-DKK)  -0.000000000000  0.000000000000 -0.004700000000
Cost/km (DKK)      -0.305500000000 18.342400000000 -0.016700000000
Horsepower         -0.109100000000  5.854200000000 -0.018600000000

In [44]:
y1 = choice_data['Fuelchoice'].values.reshape(N,J)
y1

array([[2, 1, 2, ..., 1, 1, 2],
       [2, 1, 2, ..., 1, 1, 2],
       [2, 1, 1, ..., 1, 1, 1],
       ...,
       [2, 1, 1, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 2, 2]], dtype=int64)

In [35]:
y1 = y1[:, 0] # all J elements are identical along axis=1

In [36]:
res = est.estimate(clogit.q, theta0, y1, x)

         Current function value: 4.845703
         Iterations: 13
         Function evaluations: 342
         Gradient evaluations: 55


In [37]:
cho = pd.DataFrame({v:res[v] for v in ['theta', 'se', 't']}, index=x_lab)
cho

theta             se               t
Weight (kg)         0.003772003967 0.002555404977  1.476088526432
Engine effect (kW)  0.001161655388 0.004607839797  0.252104117940
Prices (2015-DKK)   0.000000505897 0.000001897091  0.266670080504
Cost/km (DKK)      -2.063676262958 2.792358418136 -0.739044189154
Horsepower         -0.621314062076 0.428482543284 -1.450033547027

In [40]:
price_data = choice_data[['Year', 'Prices (2015-DKK)']].set_index(['Year'])
print("\nFE data for plotting:\n", price_data)

#fe_el_data = fe_data.loc['El']
d = price_data.groupby('Year')['Prices (2015-DKK)'].mean()
x = choice_data['Year'].drop_duplicates()
plt.bar(x,d)
#print("\nFE EL means:\n", fe_el_means)
#ax1.set_xlabel('Years')
#ax1.set_ylabel('km - EL', color = 'black') 
#plot_1 = ax1.plot(list(


FE data for plotting:
        Prices (2015-DKK)
Year                    
2006 403349.299100000004
2006 378504.672900000005
2006 134344.626199999999
2006 163536.214999999997
2006 150287.383200000011
...                  ...
2020 170224.371400000004
2020 303593.810400000017
2020 153689.555099999998
2020 263941.972899999993
2020 254948.742700000003

[2790 rows x 1 columns]


<BarContainer object of 15 artists>